# Проектная работа: Исследование надежности заемщиков банка

Заказчик — кредитный отдел банка. Необходимо исследовать данные и разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Откроем таблицу и изучим информацию о данных

**1. Импортируем библиотеку pandas и сохраним данные из csv-файла в датафрейм `data`, выведем первые 10 строк.**

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('C:\\Users\\jinyn\\Desktop\\data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце  days_employed обнаружены аномалии — отрицательное количество дней трудового стажа, также в столбце education данные введены разными шрифтами. При предобработке данных значения этих столбцов изменим.

**2. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Датафрейм содержит 11 колонок и 21525 срок, на этом этапе видны пропуски в колонках days_employed и total_income

### Предобработка данных

**1. Найдем и удалим пропуски.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому пропуски в этом столбце далее заполним медианным значением по каждому типу из столбца income_type.

In [5]:
# напишем цикл, чтобы перебрать категории дохода с помощью метода unique() и для каждой категории заменим пропущенные 
# значения медианным значением 
for i in data['income_type'].unique():
    mediana = data.loc[data['income_type'] == i, 'total_income'].median()
    data.loc[(data['total_income'].isna()) & (data['income_type'] == i), 'total_income'] = mediana
    
# проверим остались ли пропуски в столбце
data['total_income'].isna().sum()

0

**2. Обработаем аномальные значения.**

In [6]:
# уберем отрицательные значения в столбце days_employed
data['days_employed'] = data['days_employed'].abs()
data['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [7]:
# для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях
data.groupby('income_type')['days_employed'].median().astype('float')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов занятости (безработные и пенсионеры) получатся аномально большие значения, для дальнейшего исследования этот столбец не понадобится, поэтому оставим как есть.

In [8]:
# выведем перечень уникальных значений столбца `children`
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце есть два аномальных значения (-1 и 20), удалим эти строки из датафрейма

In [9]:
data = data[data['children'] != -1]
data = data[data['children'] != 20]

data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

**3. Продолжим удалять пропуски и изменим типы данных.** 

In [10]:
# заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type
for j in data['income_type'].unique():
    mediana = data.loc[data['income_type'] == j, 'days_employed'].median()
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == j), 'days_employed'] = mediana
    
# проверим остались ли пропуски в датасете
data.isna().sum()    

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [11]:
# заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()
data['total_income'] = data['total_income'].astype('int')

**4. Обработаем явные и неявные дубликаты.** 

In [12]:
# выведем количество строк-дубликатов
data.duplicated().sum()

54

Найдено 54 дубликата удалим их

In [13]:
data = data.drop_duplicates()

In [14]:
# приведем значения В столбце education к нижнему регистру
data['education'] = data['education'].str.lower() 
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**5. Категоризируем данные**

In [15]:
# создадим в датафрейме `data` столбец `total_income_category` с категориями с помощью метода apply, для этого 
# напишем функцию и назначим категории
def categorize_income(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'    
    return 'A'

data['total_income_category'] = data['total_income'].apply(categorize_income)

In [16]:
# выведем перечень уникальных целей взятия кредита из столбца purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Значения столбца 'purpose' также необходимо категоризировать

In [17]:
# напишем собственную функцию с именем `categorize_purpose` 
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'


data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные

**1. Исследуем зависимость между количеством детей и возвратом кредита в срок**

In [18]:
# создадим список с кол-вом детей и отсортируем его 
unique_children = sorted(data['children'].unique())  
unique_children

[0, 1, 2, 3, 4, 5]

In [19]:
# проитерируем список unique_children
for child_number in unique_children:  
    count_people = len(data[data['children'] == child_number])                         # кол-во людей с числом детей
    no_debt = len(data.loc[(data['debt'] == 0) & (data['children'] == child_number)])  # кол-во людей у которых нет долга
    have_debt = len(data.loc[(data['debt'] > 0) & (data['children'] == child_number)]) # кол-во людей у которых есть долг
    percent_have_debt = have_debt / count_people * 100                              # считаем процент должников от кол-ва людей
    
    # для сравнения выводим все переменные
    print(f'child_number: {child_number}, count_people: {count_people}, no_debt: {no_debt}, have_debt: {have_debt}, percent_have_debt: {percent_have_debt}')

child_number: 0, count_people: 14107, no_debt: 13044, have_debt: 1063, percent_have_debt: 7.535266179910682
child_number: 1, count_people: 4809, no_debt: 4365, have_debt: 444, percent_have_debt: 9.23268870867124
child_number: 2, count_people: 2052, no_debt: 1858, have_debt: 194, percent_have_debt: 9.4541910331384
child_number: 3, count_people: 330, no_debt: 303, have_debt: 27, percent_have_debt: 8.181818181818182
child_number: 4, count_people: 41, no_debt: 37, have_debt: 4, percent_have_debt: 9.75609756097561
child_number: 5, count_people: 9, no_debt: 9, have_debt: 0, percent_have_debt: 0.0


Получаем выводы:
- клиентов с детьми и имеющими просрочку всего на 2% больше, чем клиентов без детей; 
- срок возврата кредита практически не зависит от кол-ва детей, кроме клиентов с 5 детьми, но таких клиентов всего 9, поэтому для анализа не учитываем

**2. Исследуем зависимость между семейным положением и возвратом кредита в срок.**

In [20]:
# циклом пройдемся по столбцу 'family_status' и рассчитаем показатели
for status in data['family_status'].unique():
    count = len(data[data['family_status'] == status])  #посчитаем обшее кол-во людей с разными статусами
    count_debt = len(data.loc[(data['debt'] == 1) & (data['family_status'] == status)]) #посчитаем всех людей, которые не возращали кредиты в срок
    count_debt_percent = count_debt / count # найдем отношение этих велечин
    print(status,':', 'не возвращали в срок {:.2%}'.format(count_debt_percent))

женат / замужем : не возвращали в срок 7.56%
гражданский брак : не возвращали в срок 9.29%
вдовец / вдова : не возвращали в срок 6.62%
в разводе : не возвращали в срок 7.06%
Не женат / не замужем : не возвращали в срок 9.76%


Получаем вывод:  зависимость возврата кредита определенно прослеживается, не женатые/не хамужем и живущие в гражданском браке чаще не вовращают кредит в срок, а реже не возращают кредиты категория вдовец/вдова

**3. Исследуем зависимость между уровнем дохода и возвратом кредита в срок.**

In [21]:
# раннее создали столбец 'total_income_category', сгруппируем данные и рассчитаем кол-во и сумму задолжности
data_grouped = data.groupby('total_income_category').agg({'debt': ['count', 'sum']}).reset_index()

# поделим столбец с количеством должников на столбец с общим количеством клиентов и выведем
display(data_grouped, data_grouped['debt']['sum'] / data_grouped['debt']['count'] * 100)

total_income_category   debt      
                         count   sum
0                     A     25     2
1                     B   5014   354
2                     C  15938  1353
3                     D    349    21
4                     E     22     2

0    8.000000
1    7.060231
2    8.489145
3    6.017192
4    9.090909
dtype: float64

Зависимость наличия просрочки от уровня дохода прослеживается, так категории С (доход 50 тыс до 200 тыс.) и Е (доход < 30 тыс.) чаще других имеют просрочку

**4. Исследуем как разные цели кредита влияют на его возврат в срок.**

In [22]:
# ранее создали столбец, где объединили цели'purpose_category'

# проитерируем уникальные значения этого столбца
for category in data['purpose_category'].unique():  
    count = len(data[data['purpose_category'] == category]) #посчитаем обшее кол-во людей 
    count_debt = len(data.loc[(data['debt'] == 1) & (data['purpose_category'] == category)]) #посчитаем всех людей, которые не возращали кредиты в срок
    count_debt_percent = count_debt / count  # посчитаем отношение
    print(category, 'просрочили:{:.2%}'.format(count_debt_percent))


операции с недвижимостью просрочили:7.25%
операции с автомобилем просрочили:9.34%
получение образования просрочили:9.25%
проведение свадьбы просрочили:7.87%


Больше всего просрочек у кредитов на покупку автомобиля и получение образования

ОБШИЙ ВЫВОД:
1. клиенты с детьми чаще допускают просрочку кредитов, чем людей без детей, но в целом эта разница не критична, составляет всего 2%
2. количество детей у клиента не влияет на факт погашения кредита в срок
3. не женатые и живущие в гражданском браке клиенты чаще остальных групп не вовращают кредит в срок
4. клиенты с низким уровнем дохода (до 30 тыс) и с доходом от 50 до 200 тыс чаще имеют просрочку 
5. клиенты, берущие кредит на покупку автомобиля и получение образование, имеют просроченность по возврату чаще, чем кредит на другие цели

Семейное положение и количество детей клиента определенно влияют на факт погашения кредита в срок. Самые надежные заемщики это клиенты в разводе или вдовцы, без детей и с уровнем дохода от 200 до 500 тыс. рублей